In [ ]:
pip install vaderSentiment

In [ ]:
import pandas as pd
from transformers import pipeline
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import gradio as gr

In [1]:
# Load the CSV containing articles data
df_articles = pd.read_csv('american_revolution_summaries.csv')

# Initialize the Hugging Face question-answering pipeline
qa_pipeline = pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad")

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#Initializing VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

In [2]:
def get_answer_sentiment(question):
    best_answer = None
    best_score = 0

    for _, row in df_articles.iterrows():
        context = row['Summary']
        
        # Ensure the context is a string and not empty
        if not isinstance(context, str) or context.strip() == "":
            continue
        
        # Use the QA model to get an answer
        result = qa_pipeline(question=question, context=context)
        
        if result['score'] > best_score:
            best_score = result['score']
            best_answer = result['answer']

    if best_answer:
        sentiment = analyzer.polarity_scores(best_answer)
        return best_answer, sentiment
    else:
        return "No relevant content available.", {}

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Q: What should I know about the american revolution?
A: Depictions, reports, and propaganda about the event heightened tensions throughout the Thirteen Colonies


In [3]:
#Test it
question_example = "How many people died in the american revolution?"
answer = get_answer(question_example)
print(f"Q: {question_example}\nA: {answer}")

In [8]:
interface = gr.Interface(
    fn=get_answer_and_sentiment,
    inputs=gr.Textbox(label="Question", placeholder="Enter your question here..."),
    outputs=[
        gr.Textbox(label="Answer"),
        gr.JSON(label="Sentiment Analysis")
    ]
)

Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.
